In [ ]:
shp = '/mnt/g/otol_exp2/guavsforest.gdb/calibra/cos18_forest_evu'

fid = 'ffif'

refcol = 'code'

evu_cls = '17'
for_cls = '31000'

vars = [
    #'garea', 'direcoes', 'areaurban',
    #'npnts_evu', 'lenlines_evu', 'poly_evu',
    #'npnts_for', 'lenlines_for', 'denspop',
    #'pedificado_evu', 'dirbup', 'aurbbup',
    #'edievu_bup', 'tcd_mean', 'tcd_majority',
    'pop_15min'
]

out = '/mnt/g/otol_exp2/thresholds_cos18_v4.xlsx'

In [ ]:
import numpy as np
import pandas as pd
import math

from glass.rd.shp import shp_to_obj
from glass.wt import obj_to_tbl

In [ ]:
# Read data
gdf = shp_to_obj(shp)

In [ ]:
l = ['TP', 'TN', 'FP', 'FN']

fcol = [
    "threshold",
    'evu_TP', 'evu_TN', 'evu_FP', 'evu_FN',
    'evu_acc', 'evu_tpr', 'evu_tnr', 'evu_prec',
    'evu_fpr', 'evu_mcc', 'evu_f05', 'evu_f1', 'evu_f2',
    'for_TP', 'for_TN', 'for_FP', 'for_FN',
    'for_acc', 'for_tpr', 'for_tnr', 'for_prec',
    'for_fpr', 'for_mcc', 'for_f05', 'for_f1', 'for_f2'
]

In [ ]:
# Get Confusion's matrices
vars_results = []
stop = None
for v in vars:
    ftbl = []
    
    minv = int(round(gdf[v].min(), 0))
    maxv = int(round(gdf[v].max(), 0))

    # For each threshold
    for i in range(minv + 1, maxv):
        # Copy data
        dft = gdf[[fid, refcol, v]].copy()

        # Classify EVU
        dft["clsevu"] = np.where(
            dft[v] >= i, evu_cls,
            for_cls
        )

        # Classify forest
        dft["clsfor"] = np.where(
            dft[v] >= i, for_cls,
            evu_cls
        )

        # Get TP, TN, FP, FN
        dft["confevu"] = np.where(
            (dft.clsevu == evu_cls) & (dft[refcol] == evu_cls), 'TP', np.where(
                (dft.clsevu == for_cls) & (dft[refcol] == for_cls), 'TN', np.where(
                    (dft.clsevu == evu_cls) & (dft[refcol] == for_cls), 'FP', 'FN'
                )
            )
        )

        dft["conffor"] = np.where(
            (dft.clsfor == for_cls) & (dft[refcol] == for_cls), 'TP', np.where(
                (dft.clsfor == evu_cls) & (dft[refcol] == evu_cls), 'TN', np.where(
                    (dft.clsfor == evu_cls) & (dft[refcol] == for_cls), 'FN', 'FP'
                )
            )
        )

        # Get confusion in a table
        confusion = {
            "evu" : pd.DataFrame(),
            "for" : pd.DataFrame()
        }

        mesures = {
            "evu" : {},
            "for" : {}
        }
        
        confusion["evu"]['nrows'] = dft.groupby(['confevu'])[fid].nunique()
        confusion["for"]['nrows'] = dft.groupby(['conffor'])[fid].nunique()

        confusion["evu"].reset_index(inplace=True)
        confusion["for"].reset_index(inplace=True)

        #confusion["evu"]['percentage'] = (confusion["evu"].nrows * 100) / dft.shape[0]
        #confusion["for"]['percentage'] = (confusion["for"].nrows * 100) / dft.shape[0]

        for cls in confusion:
            d = {}

            for idx, row in confusion[cls].iterrows():
                try:
                    d[row[f'conf{cls}']] = row.nrows
                except:
                    print(type(confusion[cls]))
                    print(confusion[cls])
                    print(confusion[cls].columns.values)
                    stop = 1
                    break
            if stop:
                break

            for _l in l:
                if _l not in d:
                    d[_l] = 0

            acc = (d['TP'] + d['TN']) / (
                d['TP'] + d['TN'] + d['FN'] + d['FP']
            )

            try:
                tpr = d['TP'] / (d['TP'] + d['FN'])
            except:
                tpr = -99

            try:
                tnr = d['TN'] / (d['TN'] + d['FP'])
            except:
                tnr = -99

            try:
                prec = d['TP'] / (d['TP'] + d['FP'])
            except:
                prec = -99

            try:
                fpr = d['FP'] / (d['TN'] + d['FP'])
            except:
                fpr = -99

            try:
                mcc = (
                    d['TP'] * d['TN'] - d['FP'] * d['FN']
                ) / (math.sqrt(
                    (d['TP'] + d['FP']) * (d['TP'] + d['FN']) *
                    (d['TN'] + d['FP']) * (d['TN'] + d['FN'])
                ))
            except:
                mcc = -99

            try:
                f05 = ((1 + 0.5**2) * (prec * tpr)) / (0.5**2 * prec + tpr)
            except:
                f05 = -99

            try:
                f1 = (2 * prec * tpr) / (prec + tpr)
            except:
                f1 = -99

            try:
                f2= (5 * prec * tpr) / (4 * prec + tpr)
            except:
                f2 = -99

            mesures[cls]["TP"] = d["TP"]
            mesures[cls]["TN"] = d["TN"]
            mesures[cls]["FP"] = d["FP"]
            mesures[cls]["FN"] = d["FN"]
            mesures[cls]["acc"] = acc
            mesures[cls]["tpr"] = tpr
            mesures[cls]["tnr"] = tnr
            mesures[cls]["prec"] = prec
            mesures[cls]["fpr"] = fpr
            mesures[cls]["mcc"] = mcc
            mesures[cls]["f05"] = f05
            mesures[cls]["f1"] = f1
            mesures[cls]["f2"] = f2
        
        if stop:
            break

        ftbl.append([
            i,
            mesures['evu']["TP"], mesures['evu']["TN"], mesures['evu']["FP"], mesures['evu']["FN"],
            mesures['evu']["acc"], mesures['evu']["tpr"], mesures['evu']["tnr"],
            mesures['evu']["prec"], mesures['evu']["fpr"], mesures['evu']["mcc"],
            mesures['evu']["f05"], mesures['evu']["f1"], mesures['evu']["f2"],
            mesures['for']["TP"], mesures['for']["TN"], mesures['for']["FP"], mesures['for']["FN"],
            mesures['for']["acc"], mesures['for']["tpr"], mesures['for']["tnr"],
            mesures['for']["prec"], mesures['for']["fpr"], mesures['for']["mcc"],
            mesures['for']["f05"], mesures['for']["f1"], mesures['for']["f2"],
        ])

    if stop:
        break

    # Save result
    vars_results.append(pd.DataFrame(ftbl, columns=fcol))

In [ ]:
# Result to xlsx
obj_to_tbl(vars_results, out, sheetsName=vars)